In [1]:
!git clone https://github.com/lamdx4/iot-prj iot-prj

Cloning into 'iot-prj'...
remote: Enumerating objects: 110, done.
remote: Counting objects: 100% (110/110), done.
remote: Compressing objects: 100% (75/75), done.
remote: Total 110 (delta 33), reused 106 (delta 29), pack-reused 0 (from 0)
Receiving objects: 100% (110/110), 2.96 MiB | 44.62 MiB/s, done.
Resolving deltas: 100% (33/33), done.


In [2]:
!pip install -q xgboost scikit-learn imbalanced-learn pandas numpy joblib psutil

# Verify
import xgboost as xgb
print(f"XGBoost version: {xgb.__version__}")


XGBoost version: 3.0.5


In [4]:
# =========================================================
# KÉO FILE NÉN (.7z hoặc .zip) TỪ GOOGLE DRIVE -> COLAB & GIẢI NÉN
# =========================================================

from google.colab import drive
from pathlib import Path
import shutil, zipfile, sys, subprocess, os

# ️🔧 CẦN ĐIỀN ------------------------------------------------
DRIVE_ARCHIVE_PATH = "/content/drive/MyDrive/IOT prj/merged_batches.7z"  # <-- đường dẫn file .7z hoặc .zip trong Drive
DEST_DIR           = "/content/iot-prj/Data/Dataset"                              # <-- thư mục chứa file nén sau khi copy về Colab
EXTRACT_DIR        = DEST_DIR                                             # <-- nơi giải nén trên Colab
DELETE_ARCHIVE_AFTER_EXTRACT = True                                      # <-- True nếu muốn xóa file nén sau khi giải nén
# ------------------------------------------------------------

# 1) Mount Google Drive
print("🔗 Mounting Google Drive...")
drive.mount('/content/drive')

src_path     = Path(DRIVE_ARCHIVE_PATH)
dest_dir     = Path(DEST_DIR)
extract_dir  = Path(EXTRACT_DIR)

# 2) Kiểm tra file nguồn
if not src_path.exists():
    raise FileNotFoundError(
        f"Không thấy file trong Drive: {src_path}\n"
        f"• Gợi ý: kiểm tra đúng đường dẫn bắt đầu bằng /content/drive/MyDrive/ ...\n"
        f"• Lưu ý đuôi file phải là .7z hoặc .zip"
    )

# Tạo thư mục đích nếu chưa có
dest_dir.mkdir(parents=True, exist_ok=True)

# 3) Copy file nén từ Drive về Colab
local_archive = dest_dir / src_path.name
print(f"⬇️  Copy về Colab: {src_path} -> {local_archive}")
shutil.copy2(src_path, local_archive)

size_mb = local_archive.stat().st_size / (1024*1024)
print(f"✅ Đã copy: {local_archive} ({size_mb:.2f} MB)")

# 4) Giải nén
extract_dir.mkdir(parents=True, exist_ok=True)
print(f"📦 Đang giải nén vào: {extract_dir}")

suffix = local_archive.suffix.lower()

def extract_7z(archive_path: Path, out_dir: Path):
    """Giải nén .7z bằng py7zr (ưu tiên) hoặc lệnh 7z nếu muốn/ cần."""
    try:
        # Thử dùng py7zr (thuần Python)
        import py7zr
    except ImportError:
        print("⏬ Cài py7zr ...")
        # Cài đặt rồi import lại
        !pip -q install py7zr
        import py7zr

    with py7zr.SevenZipFile(archive_path, mode='r') as z:
        z.extractall(path=out_dir)

if suffix == ".zip":
    with zipfile.ZipFile(local_archive, 'r') as zf:
        zf.extractall(extract_dir)
elif suffix == ".7z":
    extract_7z(local_archive, extract_dir)
else:
    raise ValueError(f"Định dạng không hỗ trợ: {suffix}. Hỗ trợ .zip hoặc .7z")

# 5) Tuỳ chọn: xóa file nén sau khi giải nén
if DELETE_ARCHIVE_AFTER_EXTRACT:
    try:
        local_archive.unlink()
        print(f"🧹 Đã xóa file nén: {local_archive}")
    except Exception as e:
        print(f"⚠️ Không xóa được file nén: {e}")

# 6) Liệt kê vài mục sau khi giải nén
print("🗂️  Một vài file/thư mục sau giải nén:")
count = 0
for p in sorted(extract_dir.rglob('*')):
    rel = p.relative_to(extract_dir)
    print(" -", rel)
    count += 1
    if count >= 25:
        print(" ... (đã rút gọn danh sách)")
        break

print("\n🎉 Xong! Bạn có thể làm việc với dữ liệu tại:", extract_dir)


🔗 Mounting Google Drive...
Mounted at /content/drive
⬇️  Copy về Colab: /content/drive/MyDrive/IOT prj/merged_batches.7z -> /content/iot-prj/Data/Dataset/merged_batches.7z
✅ Đã copy: /content/iot-prj/Data/Dataset/merged_batches.7z (824.39 MB)
📦 Đang giải nén vào: /content/iot-prj/Data/Dataset
⏬ Cài py7zr ...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.0/97.0 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.7/51.7 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.7/142.7 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 429.9/429.9 kB 36.7 MB/s eta 0:00:00
🧹 Đã xóa file nén: /content/iot-prj/Data/Dataset/merged_batches.7z
🗂️  Một vài file/thư mục sau giải nén:
 - merged_batches
 - merged_batches/batch_01.csv
 - merged_batches/batch_02.csv
 - merged_batches/batch_03.csv
 - merged_batches/batch_04.csv
 - merged_batches/batch_05.csv
 - merged_batc

In [5]:
!python ./iot-prj/src/dataset_full/scripts/03_train_colab_highmem.py


TRAIN TWO-STAGE MODEL - COLAB PRO+ HIGH-RAM
XGBoost version: 3.0.5
🚀 GPU detected: NVIDIA L4, 23034 MiB
   💾 Current RAM usage: 0.22 GB

1. CONFIGURATION

📂 Detected Paths:
   Project root: /content/iot-prj
   Batch dir:    /content/iot-prj/Data/Dataset/merged_batches
   Model dir:    /content/iot-prj/models/full_dataset
   Stats file:   /content/iot-prj/src/dataset_full/stats/batch_statistics.json

💡 Strategy:
   Train: batch_01 + batch_04 + batch_05 (30M records)
   Test:  batch_02 sampled (1M records)
   RAM:   Expected ~33 GB (fits in 52 GB)

2. LOADING TRAINING DATA

📂 Loading batch_01... ✅ 10,000,000 records
   💾 Current RAM usage: 2.84 GB

📂 Loading batch_04... ✅ 10,000,000 records
   💾 Current RAM usage: 5.46 GB

🔧 Merging 2 batches...
✅ Training data: 20,000,000 records
   💾 Current RAM usage: 8.07 GB

3. LOADING TEST DATA

📂 Loading batch_02... ✅ 10,000,000 records

🔧 Sampling 300K from batch_02 (RAM optimized)...
✅ Test data: 300,000 records
   💾 Current RAM usage: 10.77 GB


In [23]:
!python ./iot-prj/src/dataset_full/scripts/04_evaluate_model.py

STEP 4: COMPREHENSIVE MODEL EVALUATION

📂 Paths:
   Model dir: /content/iot-prj/models/full_dataset
   Eval dir:  /content/iot-prj/results/evaluation
   Batch dir: /content/iot-prj/Data/Dataset/merged_batches

1. LOADING MODELS

📂 Found latest stage1 model: stage1_binary_20251017_115443.pkl
   Detected old format with suffix
   Timestamp: 20251017_115443

📂 Loading model files...
   ✅ Loaded: stage1_binary_20251017_115443.pkl
   ✅ Loaded: stage2_multiclass_20251017_115443.pkl
   ✅ Loaded: label_encoder_20251017_115443.pkl
   ✅ Loaded: attack_mapping_20251017_115443.pkl
   ✅ Loaded: feature_columns_20251017_115443.pkl
✅ Loaded all model artifacts
   Features: 8
   Attack types: 3

2. LOADING TEST DATA

📂 Loading test data from batch_02...
✅ Loaded 10,000,000 records

🔧 Sampling 1M for evaluation...
✅ Sampled to 1,000,000 records

3. PREPROCESSING TEST DATA
✅ Selected 8 features
🔧 Encoding categorical features (single encoder for 'proto')...
   ✅ Encoded: proto
✅ Encoded 1 categorical fe

In [14]:
!python ./iot-prj/src/dataset_full/scripts/05_visualize_results.py

GENERATE VISUALIZATIONS FROM TRAINING METRICS

1. LOADING METRICS

📂 Loading: training_metrics_20251017_085746.json
✅ Metrics loaded
   Timestamp: 20251017_085746
   Train samples: 20,000,000
   Test samples: 300,000
   Overall accuracy: 0.7508

2. PREPARING DATA
✅ Data prepared
   Categories: Normal, DDoS, DoS, Reconnaissance
   Output dir: /content/iot-prj/models/full_dataset/visualizations_20251017_085746

3. GENERATING VISUALIZATIONS

📊 1. Confusion Matrix Heatmap...
   ✅ Saved: 01_confusion_matrix.png
📊 2. Normalized Confusion Matrix...
/content/./iot-prj/src/dataset_full/scripts/05_visualize_results.py:124: RuntimeWarning: invalid value encountered in divide
  cm_normalized = cm_overall.astype('float') / cm_overall.sum(axis=1)[:, np.newaxis]
   ✅ Saved: 02_confusion_matrix_normalized.png
📊 3. Per-Category Metrics...
   ✅ Saved: 03_per_category_metrics.png
📊 4. Stage Comparison...
   ✅ Saved: 04_stage_comparison.png
📊 5. Class Distribution...
   ✅ Saved: 05_class_distribution.png


In [24]:
!zip -r /content/model.zip /content/iot-prj/models/

updating: content/iot-prj/models/ (stored 0%)
updating: content/iot-prj/models/hierarchical_overview_5pct_20251017_102314.png (deflated 23%)
updating: content/iot-prj/models/confusion_matrix_5pct_20251017_102314.png (deflated 24%)
updating: content/iot-prj/models/hierarchical_overview.png (deflated 23%)
updating: content/iot-prj/models/full_dataset/ (stored 0%)
updating: content/iot-prj/models/full_dataset/metrics_20251017_114223.json (deflated 59%)
updating: content/iot-prj/models/full_dataset/label_encoder_20251017_115443.pkl (deflated 35%)
updating: content/iot-prj/models/full_dataset/stage2_multiclass_20251017_115443.pkl (deflated 67%)
updating: content/iot-prj/models/full_dataset/stage2_20251017_085746.pkl (deflated 83%)
updating: content/iot-prj/models/full_dataset/stage2_multiclass_20251017_114223.pkl (deflated 67%)
updating: content/iot-prj/models/full_dataset/stage1_binary_20251017_114223.pkl (deflated 67%)
updating: content/iot-prj/models/full_dataset/feature_columns_20251017

In [26]:
!zip -r /content/evaluation.zip /content/iot-prj/results/evaluation/

updating: content/iot-prj/results/evaluation/ (stored 0%)
  adding: content/iot-prj/results/evaluation/evaluation_summary_20251017_094450.txt (deflated 71%)
  adding: content/iot-prj/results/evaluation/evaluation_20251017_094450.json (deflated 71%)
